# Demo Reading In Atlas Files and Electrode Localized Files

In [101]:
import sys, os
import pandas as pd
import numpy as np

from pprint import pprint
import nibabel as nb

sys.path.append('../../')
sys.path.append('../../../../')

from eztrack.edp.utils.utils import writejsonfile, loadjsonfile, MatReader
from eztrack.pipeline.utils.utils import load_results_data

In [3]:
reference = 'monopolar'
modality = 'ieeg'
center = 'umf'
patid = 'umf001'
datasetid = 'sz'

atlasname = 'dk'

results_dir = f"/Users/adam2392/Downloads/output_new/fragility/{reference}/{modality}/{center}"
outputmetafilepath = os.path.join(results_dir, f"{patid}_{datasetid}_frag.json")

atlasmapfile = "../../data/mrtrix3_labelconvert/fs2lobes_cinginc_convert.txt"
atlaslabelfile = "../../data/mrtrix3_labelconvert/fs2lobes_cinginc_labels.txt"

chlabelfile = f"/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/{patid}/elecs/{patid}_elecs_all_{atlasname}.mat"


In [4]:
# load in the data
model = load_results_data(results_dir, outputmetafilepath, datatype='frag')

patient_id = model.patient_id

# merge data
metadata = model.get_metadata()
pat_id = metadata['patient_id']
dataset_id = metadata['dataset_id'].replace("_", '')
modality = metadata['modality']

print("looking at: ", pat_id, dataset_id, modality)
print(metadata.keys())

Loading results data from:  /Users/adam2392/Downloads/output_new/fragility/monopolar/ieeg/umf/umf001_sz_fragmodel.npz
looking at:  umf001 sz ieeg
dict_keys(['ablated_contacts', 'age_surgery', 'bad_channels', 'bad_contacts', 'chanlabels', 'channeltypes', 'clinical_center', 'clinical_difficulty', 'clinical_match', 'clinical_semiology', 'dataset_id', 'date_of_recording', 'edffilename', 'engel_score', 'equipment', 'events', 'ez_elecs', 'ez_hypo_brainregion', 'ez_hypo_contacts', 'filename', 'gender', 'hand_dominant', 'highpass_freq', 'length_of_recording', 'linefreq', 'lowpass_freq', 'modality', 'non_eeg_channels', 'note', 'number_chans', 'numwins', 'offsetind', 'offsetsec', 'onset', 'onset_age', 'onsetind', 'onsetsec', 'outcome', 'patient_id', 'perturbtype', 'radius', 'reference', 'resect_elecs', 'resected_contacts', 'resultfilename', 'samplepoints', 'samplerate', 'seizure_semiology', 'stabilizeflag', 'stepsize', 'termination', 'type', 'winsize', 'onsetwin', 'offsetwin'])


In [5]:
def build_fs_atlas_map(atlasint_path, atlaslabel_path):
    """
    Build's freesurfer label name mapping from atlas names to their lobe values.
    """
    lut = {}
    with open(atlasint_path, 'r') as fd:
        # read in path line by line
        for line in fd.readlines():
            # skip comment lines
            if not line[0] == '#' and line.strip():
                val, name = line.strip().split()
                lut[name] = int(val)
                
    lut2 = {}
    with open(atlaslabel_path, 'r') as fd:
        # read in path line by line
        for line in fd.readlines():
            # skip comment lines
            if not line[0] == '#' and line.strip():
                val, name = line.strip().split()
                lut2[int(val)] = name

    masterlut = {}
    for region in lut.keys():
        masterlut[region] = lut2[lut[region]]
        
    return masterlut

In [6]:
# creates a dictionary between atlas regions and lobes
atlas_to_lobe = build_fs_atlas_map(atlasmapfile, atlaslabelfile)
display(atlas_to_lobe)

{'Unknown': 'Unknown',
 'ctx-lh-bankssts': 'L_Temporal_lobe',
 'ctx-lh-caudalanteriorcingulate': 'L_Frontal_lobe',
 'ctx-lh-caudalmiddlefrontal': 'L_Frontal_lobe',
 'ctx-lh-cuneus': 'L_Occipital_lobe',
 'ctx-lh-entorhinal': 'L_Temporal_lobe',
 'ctx-lh-fusiform': 'L_Temporal_lobe',
 'ctx-lh-inferiorparietal': 'L_Parietal_lobe',
 'ctx-lh-inferiortemporal': 'L_Temporal_lobe',
 'ctx-lh-isthmuscingulate': 'L_Parietal_lobe',
 'ctx-lh-lateraloccipital': 'L_Occipital_lobe',
 'ctx-lh-lateralorbitofrontal': 'L_Frontal_lobe',
 'ctx-lh-lingual': 'L_Occipital_lobe',
 'ctx-lh-medialorbitofrontal': 'L_Frontal_lobe',
 'ctx-lh-middletemporal': 'L_Temporal_lobe',
 'ctx-lh-parahippocampal': 'L_Temporal_lobe',
 'ctx-lh-paracentral': 'L_Frontal_lobe',
 'ctx-lh-parsopercularis': 'L_Frontal_lobe',
 'ctx-lh-parsorbitalis': 'L_Frontal_lobe',
 'ctx-lh-parstriangularis': 'L_Frontal_lobe',
 'ctx-lh-pericalcarine': 'L_Occipital_lobe',
 'ctx-lh-postcentral': 'L_Parietal_lobe',
 'ctx-lh-posteriorcingulate': 'L_Parie

In [17]:
def merge_atlas_mappings(metadata, chlabelfile, atlas_to_lobe_dict, atlasname):
    # define our matlab matfile reader
    matreader = MatReader()
    elecsdict = matreader.loadmat(chlabelfile)

    elecmatrix = elecsdict['elecmatrix']
    eleclabels = elecsdict['eleclabels']
    elecanat = elecsdict['anatomy']
    eleclabels = np.array([x[0] for x in eleclabels])
    elecanat = np.array([x[3] for x in elecanat])

    # add dictionary mapping to each contact
    ch_lobe_mapping = {}
    ch_atlas_mapping = {}
    for idx, ch in enumerate(eleclabels):
        ch_anat = elecanat[idx]

        if any(x in ch_anat for x in ['rh-insula', 'lh-insula']):
            ch_lobe = "Unknown"
        else:
            ch_lobe = atlas_to_lobe[ch_anat]

        ch_lobe_mapping[ch] = ch_lobe
        ch_atlas_mapping[ch] = ch_anat
    
    metadata['ch_lobe_mapping'] = ch_lobe_mapping
    metadata['ch_atlas_mapping'] = ch_atlas_mapping
    metadata['ch_atlas'] = atlasname
    return metadata

In [18]:
# creates a dictionary between atlas regions and lobes
atlas_to_lobe = build_fs_atlas_map(atlasmapfile, atlaslabelfile)
    
# merge in atlas mappings
metadata = merge_atlas_mappings(metadata, chlabelfile, atlas_to_lobe, atlasname)

In [19]:
print(metadata.keys())

dict_keys(['ablated_contacts', 'age_surgery', 'bad_channels', 'bad_contacts', 'chanlabels', 'channeltypes', 'clinical_center', 'clinical_difficulty', 'clinical_match', 'clinical_semiology', 'dataset_id', 'date_of_recording', 'edffilename', 'engel_score', 'equipment', 'events', 'ez_elecs', 'ez_hypo_brainregion', 'ez_hypo_contacts', 'filename', 'gender', 'hand_dominant', 'highpass_freq', 'length_of_recording', 'linefreq', 'lowpass_freq', 'modality', 'non_eeg_channels', 'note', 'number_chans', 'numwins', 'offsetind', 'offsetsec', 'onset', 'onset_age', 'onsetind', 'onsetsec', 'outcome', 'patient_id', 'perturbtype', 'radius', 'reference', 'resect_elecs', 'resected_contacts', 'resultfilename', 'samplepoints', 'samplerate', 'seizure_semiology', 'stabilizeflag', 'stepsize', 'termination', 'type', 'winsize', 'onsetwin', 'offsetwin', 'ch_lobe_mapping', 'ch_atlas_mapping', 'ch_atlas'])


In [23]:
pprint(metadata)

{'ablated_contacts': [],
 'age_surgery': 32.0,
 'bad_channels': ['event',
                  'ekgl',
                  'ekgr',
                  'lm',
                  'c65',
                  'c66',
                  'c67',
                  'c68',
                  'c69',
                  'c70',
                  'c71',
                  'c72',
                  'c73',
                  'c74',
                  'c75',
                  'c76',
                  'c77',
                  'c78',
                  'c79',
                  'c80',
                  'c81',
                  'c82',
                  'c83',
                  'c84',
                  'c85',
                  'c86',
                  'c87',
                  'c88',
                  'c89',
                  'c90',
                  'c91',
                  'c92',
                  'c93',
                  'c94',
                  'c95',
                  'c96',
                  'c97',
                  'c98',


# Convert Non-ImgPipe Localized Contacts To Similar Format

1. Convert to .mat file dictionary style and save into elecs/
2. read in .mat file 
3. add onto metadata

In [131]:
reference = 'monopolar'
modality = 'ieeg'
center = 'umg'
patid = 'umf004'
datasetid = 'sz'

atlasname = 'dk'

results_dir = f"/Users/adam2392/Downloads/output_new/fragility/{reference}/{modality}/{center}"
outputmetafilepath = os.path.join(results_dir, f"{patid}_{datasetid}_frag.json")

atlasmapfile = "../../data/mrtrix3_labelconvert/fs2lobes_cinginc_convert.txt"
atlaslabelfile = "../../data/mrtrix3_labelconvert/fs2lobes_cinginc_labels.txt"

fsdir = "/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/"
origchlabelfile = os.path.join(fsdir, f"outputfiles/{patid}/coregistration/{patid}_elec_f.mat")
chlabelfile = os.path.join(fsdir, f"{patid}/elecs/{patid}_elecs_all_{atlasname}.mat")

t1_mgz_img = os.path.join(fsdir, patid, "mri", "T1.mgz")
t1_dest_img = os.path.join(fsdir, patid,"mri", "T1.nii")
ct_src_img = os.path.join(fsdir, patid,"CT", "CT.nii")
transform_mat_path = os.path.join(fsdir, patid,"CT", "fsl_ct-to-t1fs_omat.txt")
ct_in_t1_img = os.path.join(fsdir, patid,"CT", "rCT.nii.gz")

brainmask_path = os.path.join(fsdir, patid, "mri", "brainmask.mgz")

In [109]:
# read in original ch labels file
matreader = MatReader()
origchlabels = matreader.loadmat(origchlabelfile)
elecf = origchlabels['elecf']
print(origchlabels.keys())
print(elecf.keys())

dict_keys(['__header__', '__version__', '__globals__', 'elecf'])
dict_keys(['unit', 'label', 'elecpos', 'chanpos', 'tra', 'cfg'])


In [132]:
# read in new ch labels file
matreader = MatReader()
chlabels = matreader.loadmat(chlabelfile)
elecmat_t1 = chlabels['elecmatrix']
elecanat = chlabels['anatomy']
eleclabels = chlabels['eleclabels']

eleclabels = np.array([x[0] for x in eleclabels])
elecanat = np.array([x[3] for x in elecanat])

[-30.7511104   24.45556607 -10.93617317]


In [6]:
eleclabels = np.array(elecf['label'])
elecmat = np.array(elecf['elecpos'])

print(eleclabels.shape, elecmat.shape)

(146,) (146, 3)


In [17]:
os.listdir("/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/la03/CT/")

['CT.nii']

In [20]:
# convert mgz to nifti images
# convert_mgz_to_nifti(t1_mgz_img, t1_dest_img)

# run fsl coregistration CT->T1 nifti
map_ct_to_t1(ct_src_img, t1_dest_img, ct_in_t1_img, transform_mat_path)

In [103]:
import subprocess
from ast import literal_eval

def convert_mgz_to_nifti(src_img, dest_img):
    cp = subprocess.run("mrconvert %s %s --force;" \
                            % (src_img, dest_img),
                        shell=True, stdout=subprocess.PIPE)
    return cp
    
def map_ct_to_t1(ct_src_img, t1_dest_img, ct_in_t1_img, transformat_mat_path):
    fsl_str_cmd = f"flirt -in {ct_src_img} \
                            -ref {t1_dest_img} \
                            -omat {transformat_mat_path} \
                            -out {ct_in_t1_img}"
    print(fsl_str_cmd)
    cp = subprocess.run(fsl_str_cmd, shell=True, stdout=subprocess.PIPE)
    
    
    return cp
    
def transform_ct_to_t1(ctcoords, t1_dest_img, ct_src_img, transform_mat):
    coords_str = " ".join([str(x) for x in ctcoords])
    
    cp = subprocess.run("%s | img2imgcoord -mm -src %s -dest %s -xfm %s" \
                            % (coords_str, ct_src_img, t1_dest_img, transform_mat),
                        shell=True, stdout=subprocess.PIPE)
    transformed_coords_str = cp.args.strip().split("|")[0]
    return np.array([float(x) for x in transformed_coords_str.split(" ") if x])

from nibabel.affines import apply_affine

def filter_electrodes_outbrain(elec_coords_mm, brainmask_img):
    brainmask_data = brainmask_img.get_fdata()
    brainmask_affine = brainmask_img.affine
    inv_affine = np.linalg.inv(brainmask_affine)

    # Convert contact xyz coordinates to CT voxels
    elec_coords_CTvox = {}
    for label, contact in elec_coords_mm.items():
        elec_coords_CTvox[label] = apply_affine(inv_affine, contact)

    # Filter out electrodes not within brain mask
    elec_in_brain = {}
    for label, contact in elec_coords_CTvox.items():
        if brainmask_data[int(contact[0]), int(contact[1]), int(contact[2])] != 0:
            elec_in_brain[label] = contact
    return elec_in_brain

In [123]:
# transform ct to t1
elecmat_t1 = []
for elecxyz in elecmat:
    elecmat_t1.append(transform_ct_to_t1(elecxyz, t1_dest_img, ct_src_img, transform_mat_path))
elecmat_t1 = np.array(elecmat_t1)
print(elecmat_t1.shape)
print(eleclabels.shape)

(66, 3)
(66,)


In [133]:
elecdict = {}
for i, chlabel in enumerate(eleclabels):
    elecdict[chlabel] = elecmat_t1[i]

In [135]:
brainmask_img = nb.load(brainmask_path)
in_contacts = filter_electrodes_outbrain(elecdict, brainmask_img)
out_contacts = [ch for ch in eleclabels if ch not in in_contacts]
print(in_contacts.keys())
print(len(in_contacts))

dict_keys(['ltp1', 'ltp2', 'ltp3', 'la1', 'la2', 'la3', 'la4', 'la5', 'la6', 'lofs1', 'lofs2', 'lofs3', 'lph1', 'lph2', 'lh1', 'lh2', 'lh3', 'lmcsm1', 'lmcsm2', 'ra1', 'ra2', 'ra3'])
22
